In [1]:
pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 802.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 376.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 KB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 65.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.2 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=83b8009fe64f8feb47bf2b044912f281b65c8a39e7c06fc057ea6c2a295404a3
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/

In [2]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments

In [3]:
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/content/drive/MyDrive/class/language/checkpoint-paircls",
    max_seq_length=64,
)

downstream_model_checkpoint_fpath: /content/drive/MyDrive/class/language/checkpoint-paircls/epoch=0-val_loss=0.80.ckpt


In [4]:
from transformers import BertTokenizer


In [5]:
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [6]:
import torch

In [7]:
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location = torch.device('cpu')
)

In [8]:
from transformers import BertConfig

In [20]:
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel()
)

In [21]:
fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel()

3

In [ ]:
pretrained_model_config

In [23]:
from transformers import BertForSequenceClassification

In [24]:
model = BertForSequenceClassification(pretrained_model_config)

In [25]:
model.load_state_dict({k.replace("model.","") : v for k,v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

In [ ]:
fine_tuned_model_ckpt['state_dict'].items()

In [ ]:
model.eval()

In [31]:
def inference_fn(premise, hypothesis) :
  inputs = tokenizer(
      [(premise, hypothesis)],
      max_length = args.max_seq_length,
      padding="max_length",
      truncation = True,
      return_tensors = "pt"
  )

  with torch.no_grad() :
    outputs = model(**inputs)
    prob = outputs.logits.softmax(dim=-1)
    entailment_prob = round(prob[0][0].item(), 2)
    contradiction_prob = round(prob[0][1].item(), 2)
    neutral_prob = round(prob[0][2].item(), 2)

    if torch.argmax(prob)==0 :
      pred = "참 (entailment)"
    elif torch.argmax(prob)==1 :
      pred = "거짓 (contradiction)"
    else :
      pred = "중립 (neutral)"

  return {  
      'premise' : premise,
      'hypothesis' : hypothesis,
      'entailment_data' : f'참 {entailment_prob}',
      'contradiction_data' :f'거짓 {contradiction_prob}',
      'neutral_data': f'중립 {neutral_prob}',
      'entailment_width' : f'{entailment_prob * 100}%',
      'contradiction_width' : f'{contradiction_prob * 100}%',
      'neutral_width': f'{neutral_prob * 100}%'
  }


     

In [32]:
!mkdir /root/.ngrok2 && echo "authtoken: 2KUKWK7BGUqKWYtcmI6OA1BQHR3_73E9abYcCoNbPq4kqDvJb" > /root/.ngrok2/ngrok.yml

In [33]:
from ratsnlp.nlpbook.paircls import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()


 * Serving Flask app "ratsnlp.nlpbook.paircls.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://8c26-34-124-177-134.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [19/Jan/2023 04:45:17] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Jan/2023 04:45:17] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Jan/2023 04:45:40] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Jan/2023 04:46:11] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Jan/2023 04:46:30] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Jan/2023 04:47:51] "POST /api HTTP/1.1" 200 -
